**Πριν ξεκινήσεις να τρέχεις τον κώδικα**

Κάνε κλικ στο <kbd>Runtime</kbd> στην κορυφή της σελίδας, και στις επιλογές <kbd>change runtime type</kbd> ή <kbd>runtime options</kbd>. Εκεί επίλεξε ένα κουτάκι που λέει <kbd>(T4) GPU</kbd>.

**Πώς χρησιμοποιείς αυτό το αρχείο;**

Αυτό το αρχείο είναι ένα *jupyter notebook*, που αποτελείται από κελιά κειμένου (όπως αυτό) και κελιά κώδικα (όπως το παραπάνω). Μπορείς να τρέξεις τα κελιά κώδικα με <kbd>shift</kbd> + <kbd>enter</kbd>. Πριν προχωρήσεις, είναι σημαντικό να τρέξεις το παραπάνω κελί, ώστε o υπολογιστής να μπορεί να εισάγει όλες τις απαραίτητες συναρτήσεις.


In [1]:
import pip

import numpy as np
import os
from glob import glob
import torch
from PIL import Image
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import torch.nn.functional as F

%pip install medmnist

%pip install SimpleITK
import SimpleITK as sitk


!git clone https://github.com/clarastegehuis/machine_learning_medical_data_workshop/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 21.8 MB/s eta 0:00:00
Cloning into 'machine_learning_medical_data_workshop'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116 (from 1)
Receiving objects: 100% (116/116), 29.33 MiB | 27.23 MiB/s, done.
Resolving deltas: 100% (62/62), done.


# Τα μαθηματικά πίσω από την ΤΝ - Εφαρμογή: ιατρικά δεδομένα

Τα νευρωνικά δίκτυα (CNNs) μπορούν να χρησιμοποιηθούν για την αυτόματη ερμηνεία εικόνων. Οι υπολογιστές είναι μάλιστα καλύτεροι σε ορισμένες εργασίες ανάλυσης εικόνας από τους ανθρώπους, επειδή οι άνθρωποι έχουν μικρή διάρκεια συγκέντρωσης. Σ' αυτό το εργαστήριο θα ρίξουμε μια ματιά στο πώς ένας υπολογιστής μπορεί να ερμηνεύει εικόνες. Θα εργαστούμε με ένα (δημόσιο) ιατρικό σύνολο δεδομένων και θα εκπαιδεύσουμε το δικό μας δίκτυο που μπορεί να ξεχωρίζει μεταξύ εικόνων δοντιών με και χωρίς απονεύρωση.

### Ψηφιακές εικόνες

Ο υπολογιστής βλέπει μια εικόνα ως έναν μεγάλη πίνακα με αριθμούς, κάθε στοιχείο του πίνακα (γνωστότερο ως *pixel*) περιέχει την τοπική ένταση της εικόνας. Στην περίπτωση μιας έγχρωμης εικόνας, πρόκειται για τρεις πίνακες τον έναν πάνω στον άλλο, που αντιπροσωπεύουν αντίστοιχα το κόκκινο, το μπλε και το πράσινο κανάλι. Στην περίπτωση μιας ασπρόμαυρης εικόνας, η ψηφιακή εικόνα είναι ένας μοναδικός πίνακας με εντάσεις.

In [2]:
#definieert twee functies die je later kan gebruiken om de images te openen en te visualiseren
def open_img(path):
    if path.endswith('.png'):
        return np.array(Image.open(path).convert('L'))
    elif path.endswith('.mhd'):
        return sitk.GetArrayFromImage(sitk.ReadImage(path))[32,:,:] # return 1 slice of the image

def visualize(img, clim=[-300,450]):
    plt.imshow(img, cmap='gray', clim=clim)
    plt.axis('off')
    plt.show()

In [ ]:
# inladen van beeld van de ribbenkast
# definieer pad naar beeld
# img_path = 'data/ribs/VinDr_RibCXR_train_000.png'
img_path = '/content/machine_learning_medical_data_workshop/TEV1P1CTI.mhd'
img = open_img(img_path)
# visualiseer beeld
visualize(img)

Uit hoeveel pixels bestaat dit beeld? np.shape(img) laat de vorm (aantal rijen en kolommen) van de matrix zien.

In [ ]:
print(np.shape(img))

# Hoe veel pixels zijn dit?

### Convoluties
Een computer kan een beeld begrijpen door middel van zogenaamde convoluties. Een convolutie bestaat altijd uit een *kernel*, een kleine matrix met daarin een kenmerkend patroon, die lokaal wordt vermenigvuldigd met de beeldintensiteiten. In de onderstaande animatie is het bovenste groene vlak de kernel, en het blauwe vlak het te interpreteren beeld, het schuiven van de kernel noemen we de convolutie. Het resultaat van een convolutie is nog steeds een *matrix*, die vergelijkbare afmetingen heeft als het originele beeld.

![](https://upload.wikimedia.org/wikipedia/commons/0/04/Convolution_arithmetic_-_Padding_strides.gif?20190413174630)

 Met een convolutie wordt in feite de intensiteit van iedere pixel vergeleken met die van zijn buren, afhankelijk van het patroon in de kernel. Door het patroon in de kernel slim te kiezen, kunnen bepaalde features in het beeld worden opgepikt, bijvoorbeeld verticale randen. Effectief wordt er per pixel gekeken hoe zijn omgeving matcht met het patroon in de kernel. Daarnaast kan een convolutie gebruikt worden om ruis in een beeld te verminderen, dit noemen we *smoothing*.

 Hieronder laten we een paar voorbeelden van convolutiekernels zien, aan jullie om te beschrijven wat voor effect ze hebben op het beeld.

In [ ]:
# dit is een functie die een convolution toepast op een beeld
def apply_conv(image, kernel, iter=1):
    image, kernel = torch.from_numpy(image).float(), torch.from_numpy(kernel).float()
    img_shape, kernel_shape = image.shape, kernel.shape
    fig, ax = plt.subplots(1,1)
    for level in range(iter):
        image = F.conv2d(image.reshape(1,1, img_shape[0], img_shape[1]),
                         kernel.reshape(1,1, kernel_shape[0], kernel_shape[1]),
                         padding='same').squeeze()
        ax.imshow(image.numpy(), cmap='gray', clim=[-300,450])
        ax.set_title(f'Applied convolution {level+1} times')
        display(fig)
        clear_output(wait=True)
        plt.pause(0.1)
        plt.close()

Nu kun je zelf een kernel loslaten op het beeld van de ribben.

**Vraag:**
Wat gebeurt er met het beeld door de convolutie? En wat gebeurt er als je meerdere convoluties achter elkaar toepast?

In [ ]:
# convolutie 1: detecteren verschuiven naar links
# eerst kernel definieren:
kernel = np.array([[0, 0, 0],
                   [0, 0, 1],
                   [0, 0, 0]])
# hoe vaak willen we de convolutie toepassen?
n_iters = 20

apply_conv(img, kernel, n_iters)

De tweede convolutie veschuift het beeld naar boven.

**Vraag:** Kun je ook een convolutie maken die het beeld naar rechts beneden schuift (schuin)?

In [ ]:
# convolutie 2: verschuiven naar boven
# eerst kernel definieren:
kernel = np.array([[0, 0, 0],
                   [0, 0, 0],
                   [0, 1, 0]])
# hoe vaak willen we de convolutie toepassen?
n_iters = 20

apply_conv(img, kernel, n_iters)

Als het goed is heb je hierboven begrepen hoe je een beeld naar rechts, links, boven of onder kan verplaatsen, of schuin kan verplaatsen. Als je de kernel groter maakt, kun je je beeld ook ingewikkeldere sprongen laten maken.

**Vraag:**
Kun je bijvoorbeeld met een $5 \times 5$ kernel het beeld een paardensprong laten maken? (2 pixels omhoog, en 1 naar links)? Nu staan er alleen maar nullen in de kernel, dus je moet 1 of meerdere vakjes aanpassen.

In [ ]:
# Opdracht: maak een convolutie die het beeld een paardensprong laat maken: i
kernel = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]])

n_iters = 20

apply_conv(img, kernel, n_iters)

De convolutie hieronder past 'smoothing' toe, en maakt het beeld onscherper.

**Vraag:** Wat gebeurt er als je de gewichten in de kernel iets aanpast?

In [ ]:
# convolutie 3: Een convolutie die 'smoothing' toepast: Voor iedere pixel neemt de convolution het gewogen gemiddelde van de pixelwaarde en de pixelwaarden van de 8 omliggende pixels. Wat gebeurt er als je de gewichten van de kernel aanpast?
kernel = np.array([[1, 2, 1],
                   [2, 4, 2],
                   [1, 2, 1]]) * 1/16

n_iters = 20

apply_conv(img, kernel, n_iters)

De onderstaande convolutie hadden we eerder al gezien, die vindt verticale lijnen. Deze convolutie hoef je maar één keer toe te passen, omdat je in een keer alle lijnen kunt vinden.

**Vraag:** Kun je ook een convolutie maken die horizontale lijnen vindt? En wat gebeurt er als je de 1 en en -1 en omdraait?

In [ ]:
# convolutie 3: detecteren van verticale lijnen
kernel = np.array([[1, 0, -1],
                   [1, 0, -1],
                   [1, 0, -1]])

n_iters = 1

apply_conv(img, kernel, n_iters)


# opdracht:kun je ook een kernel maken die horizontale randen vindt?

## Convolutional Neural Network
Deze convolutieoperaties vormen de basis van een zogenaamd *convolutional neural network*. In principe is dit een neuraal netwerk dat kan worden ingezet voor allerlei computer vision taken, zoals het classificeren van beelden, objecten detecteren of zelfs de precieze grenzen van een object vinden in een beeld. Convolutional neural networks bestaan uit een stapeling van convoluties. Door convoluties vaker toe te passen in een 'stapel', kan de computer een groeiende lokale regio rondom elke pixel kan bekijken (perceptive field).
De kernels in al deze convoluties worden niet door mensen bepaald, maar worden bepaald tijdens het *trainen* van dit netwerk. In de rest van dit notebook laten we een klein voorbeeld zien van hoe dit werkt en hoe we kunnen bepalen hoe goed dit netwerk is in zijn taak.

**De taak:**

We maken gebruik van de zogenaamde pneumonia dataset. Deze bevat gedownsamplede röntgenfoto's van de borstkas, van zowel gezonde patienten als van patienten met een longontsteking. We gaan een neuraal netwerk trainen dat automatisch voor een dergelijk beeld kan bepalen of er sprake is van longontsteking of niet.

In [ ]:
# downoald de dataset met foto's van de longen
import medmnist
%pip install monai

dataset = medmnist.PneumoniaMNIST(split="train", download=True)

In [ ]:
import monai
# maakt een custom dataset class die de data in de juiste vorm geeft

class MedMNISTData(monai.data.Dataset):

    def __init__(self, datafile, transform=None):
        self.data = datafile
        self.transform = transform


    def __getitem__(self, index):
        # Make getitem return a dictionary with keys ['img', 'label'] for the image and label respectively
        image = torch.from_numpy(np.array(self.data[index][0])).float()
        if self.transform:
            image = self.transform(image)
        return {'img': image, 'label': self.data[index][1]}

    def __len__(self):
        return len(self.data)

In [ ]:
# Een functie die de foto's uit de dataset visualiseert
def visualize_sample(sample):
    plt.imshow(sample['img'], 'gray')
    if sample['label'] == 1:
        plt.title('Patient with pneumonia')
    else:
        plt.title('Healthy patient')
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
# Intensiteiten normaliseren voor het netwerk straks en de training data maken
from monai.transforms import NormalizeIntensity

data_transform = NormalizeIntensity(subtrahend=.5, divisor=.5)

train_dataset = MedMNISTData(dataset, transform=data_transform)

### Vraag 1 (doel: experimenteren met code en dataset verkennen):
train_dataset is een verzameling van 4708 scans waarin elke scan een label heeft, deze kan 0 of 1 zijn. Label 0 betekent dat de patient gezond is, en label 1 betekent dat de patient longontsteking heeft , Als je <kbd>visualize(train_dataset[k][‘img’])</kbd> intoetst dan kun je het $k$-de plaatje zien, en als je <kbd>print(train_dataset[k][‘label’])</kbd> intoetst dan zie je de label die bij dat plaatje hoort. Experimenteer met <kbd>visualize_sample(train_dataset[k])</kbd> om de dataset te verkennen.


Hieronder kun je een foto de longen van een willekeurige patient laten zien. Om de voorspellingen van het algoritme later beter te kunnen interpreteren, is het belangrijk om te weten hoe veel foto's er in de data zitten van gezonde longen, en hoe veel er van ongezonde longen inzitten.

Hieronder kun je kijken hoe veel data uit de ene, en uit de andere klasse komen.

**Vraag:** Is de dataset gebelanceerd? Hoe veel procent van de trainingsdata gaat over patienten met een longontsteking?

In [ ]:
# visualiseer een random sample
index = np.random.choice(np.arange(len(train_dataset)))
visualize_sample(train_dataset[index])

#Vraag: is deze dataset imbalanced?
counts = {0: 0, 1:0}
for sample in train_dataset:
    counts[sample['label'][0]] += 1
print('Aanal label 0:', counts[0], 'Aantal label 1:', counts[1])

Nu gaan we het echte netwerk aanmaken. We splitsen de data in een testset en een validatieset om later te kunnen kijken of het model niet overfit. We maken een model met twee lagen van convoluties van 3x3 kernels. Daarachter komt een neuraal netwerk, en we zorgen dat er één output is (wel of niet longontsteking). Dit model staat geprogrammeerd als Net().


**Vraag:**
Wat is de receptive field van dit netwerk (hoe veel pixels)?

In [ ]:
# validatiedataset aanmaken, om te kijken hoe het model generaliseert tijdens het trainen. De validatieset wordt niet gebruikt om de gewichten van het model op te fitten.
val_dataset = MedMNISTData(medmnist.PneumoniaMNIST(split='val', download=False))

# dataloader die de data inlaadt voor het trainen
train_dataloader = monai.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = monai.data.DataLoader(val_dataset, batch_size = 32, shuffle=False)

In [ ]:
# Hier wordt het echte model gedefinieerd. Dit model is een convolutioneel neuraal netwerk (CNN) dat bestaat uit 2 convolutionele lagen en 2 fully connected lagen.
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)             # Een convolutie met 1 input channel (de afbeelding), 32 output channels (32 verschillende kernels), 3x3 pixel kernel
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)            # Een tweede convolutie met 32 input channels (de 32 output channels van de vorige laag), 64 output channels (verschillende kernels), 3x3 pixel kernel
        self.fc1 = nn.Linear(in_features=9216, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=1)                                       # De output laag met 1 output neuron (de voorspelling, tussen 0 (geen longontsteking) en 1( longontsteking) )

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        output = self.fc2(x)
        return output

net = Net()


We moeten ook bepalen wat de loss functie is die het netwerk gebruikt als we gaan trainen. We gebruiken de 'binary cross-entropy loss' die we besproken hadden.

In [ ]:
model = Net()
model.cuda() # op de GPU zetten, zodat het trainen sneller gaat


optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
# loss functie: Binary cross entropy (want classificatie).
loss_function = torch.nn.BCEWithLogitsLoss()

In [ ]:
from tqdm import tqdm
# functie om het model te trainen

def train_medmnist(model, train_dataloader, val_dataloader, optimizer, epochs, device='cuda', val_freq=1):
    train_loss = []
    val_loss = []

    for epoch in tqdm(range(epochs)):
        # model in train modus
        model.train()
        steps = 0
        epoch_loss = 0
        # loop over de batches in training data
        for batch in train_dataloader:
            optimizer.zero_grad()
            images = batch['img'].float().to(device)
            labels = batch['label'].float().to(device)
            # haal plaatjes door het model
            output = model(images.unsqueeze(1))
            # bereken de loss tussen de targets en de outputs van het model
            loss = loss_function(output, labels)
            epoch_loss += loss.item()
            # back propagation, update de weights in het netwerk
            loss.backward()
            optimizer.step()
            steps += 1

        train_loss.append(epoch_loss/steps)

        # validation loop
        if epoch % val_freq == 0:
            steps = 0
            val_epoch_loss = 0
            model.eval()
            for batch in val_dataloader:
                images = batch['img'].float().to(device)
                labels = batch['label'].float().to(device)
                output = model(images.unsqueeze(1))
                loss = loss_function(output, labels)
                val_epoch_loss += loss.item()
                steps += 1
            val_loss.append(val_epoch_loss/steps)

    # plot the losses together
    plt.plot(train_loss, label='train loss')
    plt.plot(np.arange(0, epochs, val_freq), val_loss, label='val loss')
    plt.legend()
    plt.show()

    return model, train_loss, val_loss

Nu we het neurale netwerk, de loss functie, de data geïntroduceerd is, kunnen we het model echt gaan trainen, en de kernels leren die het model gaat gebruiken om gezonde en ongezonde patiënten te onderscheiden. De code hieronder laat zien hoe de loss functie naar beneden gaat tijdens de training, op zowel te trainingsdata als de validatiedata.


**Vraag:** Als je de plots van de loss functie op de trainingsdata en op de validatiedata ziet, is het model dan overfitted denk je? En nu hebben we het model 100 epochs getraind. Had dit ook korter gekund?

In [ ]:
val_freq = 10

# 100 iteraties trainen
n_epochs = 100
model, train_loss, val_loss = train_medmnist(model, train_dataloader, val_dataloader, optimizer, epochs=n_epochs, val_freq=val_freq)

We hebben hierboven gezien wat specifieke kernels kunnen herkennen in een beeld. Maar wat heeft ons algoritme voor kernels gevonden om te classificeren tussen gezonde en zieke longen? Hieronder zien we de kernels van de tweede laag van convoluties. Omdat de eerste laag al 16 convoluties heeft gedaan, zijn er 16 verschillende lagen van kernels in de tweede laag. Je kunt ze allemaal bekijken door de variabele <kbd>input_index</kbd> aan te passen.

In [ ]:
input_index = 16
fig, axs = plt.subplots(8,8, layout='constrained')
for i in range(64):
    kernel = model.conv2.weight[i,input_index,:,:].detach().cpu().numpy()
    cur_ax = np.unravel_index(i, [8,8])
    s = axs[cur_ax].imshow(kernel, clim=[-0.1,0.1],cmap = 'Greys')
    axs[cur_ax].axis('off')
plt.suptitle(f'Geleerde kernels uit de tweede convolutional laag, input channel {input_index}')
plt.tight_layout()
plt.show()


En hier zien we ook de kernels uit de eerste laag van convoluties

In [ ]:
print(np.shape(model.conv1.weight))
fig, axs = plt.subplots(4,8, layout='constrained')
for i in range(32):
    kernel = model.conv1.weight[i,0,:,:].detach().cpu().numpy()
    cur_ax = np.unravel_index(i, [4,8])
    s = axs[cur_ax].imshow(kernel, clim=[-0.1,0.1],cmap = 'Greys')
    axs[cur_ax].axis('off')
plt.suptitle(f'Geleerde kernels uit de eerste convolutional laag')
plt.tight_layout()
plt.show()

## Performance assessment

Nu we het model getraind hebben, gaan we kijken in hoeverre dit model goede voorspellingen kan doen. Eerst bepalen we de recall en precision van het model. De recall vertelt ons hoeveel van de positieven er gemist worden door het model (vals negatieven). De precisie meet hoeveel van de positief geclassificeerde samples daadwerkelijk positieve samples zijn (vals positieven). Welke maat belangrijker is, is afhankelijk van het probleem. Bij het detecteren van een extreem zeldzame vorm van kanker heb je bijvoorbeeld het liefst een hoge recall en accepteer je daarmee een lagere precisie. Het is beter om de daadwerkelijke positieven wél te detecteren en daarmee in een vervolgonderzoek de vals positieven eruit te filteren, dan de positieven compleet te missen.
We gebruiken de test dataset (dus niet de validatiedataset) om deze metrics te bepalen.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/525px-Precisionrecall.svg.png)

Voordat we deze metrics gaan bepalen, bekijken we eerst een paar outputs van het model.


In [ ]:
def validation_results_visualize(model, dataset):
    index = np.random.randint(0, len(dataset))
    image = dataset[index]['img']
    plt.imshow(image.numpy().squeeze(), cmap='gray')
    image = image.float().to('cuda')
    label = dataset[index]['label'].item()
    with torch.no_grad():
        output = F.sigmoid(model(image.view(1,1,28,28))).squeeze()
    plt.yticks([])
    plt.xticks([])
    plt.title(f'Echte waarde: {label}, voorspelling model: {int(output)}')
    plt.show()

In [ ]:
for i in range(10):
    validation_results_visualize(model, val_dataset)

In [ ]:
# deze functien berekent de precision en recall van het model
def get_precision_recall(model, dataloader):
    model.eval()
    TP, TN, FP, FN = 0, 0, 0, 0
    total = 0
    for data in dataloader:
        images = data['img'].float().to('cuda')
        labels = data['label'].squeeze()
        total += len(labels)
        with torch.no_grad():
            output = F.sigmoid(model(images.unsqueeze(1))).squeeze().cpu()
        pred_classes = (output >= 0.5).to(torch.int8)
        TP += (pred_classes * labels).sum()
        TN += ((1 - pred_classes) * (1 - labels)).sum()
        FP += (pred_classes * (1 - labels)).sum()
        FN += ((1 - pred_classes) * labels).sum()
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return precision, recall

In [ ]:
test_dataset = MedMNISTData(medmnist.PneumoniaMNIST(split='test', download=False))
test_loader = monai.data.DataLoader(test_dataset, batch_size = 32, shuffle=False)

precision, recall = get_precision_recall(model, test_loader)
print(f'De precision van het getrainde model is {precision:.2f}, de recall van het getrainde model is {recall:.2f}.')

**Vraag:** Wat vind je van de precisie en recall van het model (ook denkend aan hoe veel procent van de patienten in de data longontsteking hebben?)

Hieronder kun je een zogenaamde 'confusion matrix' zien. Hoe veel longontstekingen zijn over het hoofd gezien? En hoe vaak wordt er een patient onterecht gediagnostiseerd met longontsteking? Welk beeld geeft dit van hoe krachtig het model is? Kun je er ook achterkomen of het model ook goed werkt op de validatiedataset? De dataloader voor de validatieset heet <kbd>val_dataloader</kbd>.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model,dataloader):
    """
    Plots a confusion matrix using true and predicted labels.

    :param y_true: List or array of true labels.
    :param y_pred: List or array of predicted labels.
    :param labels: List of label names (optional).
    """
    model.eval()
    true_labels = []
    predicted_labels = []


    for data in dataloader:
        images = data['img'].float().to('cuda')
        labels = data['label'].squeeze()
        with torch.no_grad():
            output = F.sigmoid(model(images.unsqueeze(1))).squeeze().cpu()
        pred_classes = (output >= 0.5).to(torch.int8)
        true_labels.extend(labels.numpy())
        predicted_labels.extend(pred_classes.numpy())

    cm = confusion_matrix(true_labels, predicted_labels, labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['geen longontsteking', 'longontsteking'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

plot_confusion_matrix(model, test_loader)